# GCN practice code

- import basic library

In [83]:
import torch

torch.version

<module 'torch.version' from 'e:\\conda3\\envs\\test\\lib\\site-packages\\torch\\version.py'>

In [84]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

import networkx as nx
import numpy as np
from random import randint, expovariate
import matplotlib.pyplot as plt

- Generate the network

In [85]:
S_CPU_MAX = []
S_BW_MAX = []

# 随机生成一个图(20节点, 100链路)
net = nx.gnm_random_graph(n=20, m=100)

# 设置所有节点的CPU数据, 并同时统计最大值最小值
min_cpu_capacity = 1.0e10
max_cpu_capacity = 0.0
for node_id in net.nodes:
    net.nodes[node_id]['CPU'] = randint(50, 100)
    net.nodes[node_id]['LOCATION'] = randint(0, 2)
    if net.nodes[node_id]['CPU'] < min_cpu_capacity:
        min_cpu_capacity = net.nodes[node_id]['CPU']
    if net.nodes[node_id]['CPU'] > max_cpu_capacity:
        max_cpu_capacity = net.nodes[node_id]['CPU']

# 设置链路的带宽数据, 并同时统计最大带宽最小带宽
min_bandwidth_capacity = 1.0e10
max_bandwidth_capacity = 0.0
for edge_id in net.edges:
    net.edges[edge_id]['bandwidth'] = randint(50, 100)
    if net.edges[edge_id]['bandwidth'] < min_bandwidth_capacity:
        min_bandwidth_capacity = net.edges[edge_id]['bandwidth']
    if net.edges[edge_id]['bandwidth'] > max_bandwidth_capacity:
        max_bandwidth_capacity = net.edges[edge_id]['bandwidth']

# data=True: 返回的是 NodeDataView 对象, 该对象不仅包含每个顶点的 ID 属性, 还包括顶点的其他属性
for s_node_id, s_node_data in net.nodes(data=True):
    S_CPU_MAX.append(s_node_data['CPU'])

# 统计每个底层节点周围链路带宽和
for s_node_id in range(len(net.nodes)):
    total_node_bandwidth = 0.0
    for link_id in net[s_node_id]:
        total_node_bandwidth += net[s_node_id][link_id]['bandwidth']
    S_BW_MAX.append(total_node_bandwidth)


In [86]:
 # S_CPU_Free
s_CPU_remaining = []
s_bandwidth_remaining = []

# 1 表示目前哪些节点被占用, 0 相反
current_embedding = [0] * len(net.nodes)

# 节点剩余资源
for s_node_id, s_node_data in net.nodes(data=True):
    s_CPU_remaining.append(s_node_data['CPU'])
    
# 节点周围剩余带宽资源
for s_node_id in range(len(net.nodes)):
    total_node_bandwidth = 0.0
    for link_id in net[s_node_id]:
        total_node_bandwidth += net[s_node_id][link_id]['bandwidth']
    s_bandwidth_remaining.append(total_node_bandwidth)

In [87]:
# 底层网络特征矩阵
substrate_features = []
substrate_features.append(S_CPU_MAX)
substrate_features.append(S_BW_MAX)
substrate_features.append(s_CPU_remaining)
substrate_features.append(s_bandwidth_remaining)
substrate_features.append(current_embedding)

print(substrate_features)

[[99, 89, 67, 75, 69, 76, 85, 76, 62, 68, 53, 100, 90, 87, 99, 65, 65, 82, 73, 76], [1033.0, 803.0, 367.0, 660.0, 634.0, 574.0, 638.0, 903.0, 872.0, 799.0, 896.0, 902.0, 1103.0, 632.0, 770.0, 857.0, 933.0, 159.0, 1101.0, 498.0], [99, 89, 67, 75, 69, 76, 85, 76, 62, 68, 53, 100, 90, 87, 99, 65, 65, 82, 73, 76], [1033.0, 803.0, 367.0, 660.0, 634.0, 574.0, 638.0, 903.0, 872.0, 799.0, 896.0, 902.0, 1103.0, 632.0, 770.0, 857.0, 933.0, 159.0, 1101.0, 498.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [88]:
substrate_features = torch.tensor(substrate_features)
print(substrate_features)
print(substrate_features.shape)

# transpose: 转置
substrate_features = torch.transpose(substrate_features, 0, 1)
print(substrate_features)
print(substrate_features.shape)

tensor([[  99.,   89.,   67.,   75.,   69.,   76.,   85.,   76.,   62.,   68.,
           53.,  100.,   90.,   87.,   99.,   65.,   65.,   82.,   73.,   76.],
        [1033.,  803.,  367.,  660.,  634.,  574.,  638.,  903.,  872.,  799.,
          896.,  902., 1103.,  632.,  770.,  857.,  933.,  159., 1101.,  498.],
        [  99.,   89.,   67.,   75.,   69.,   76.,   85.,   76.,   62.,   68.,
           53.,  100.,   90.,   87.,   99.,   65.,   65.,   82.,   73.,   76.],
        [1033.,  803.,  367.,  660.,  634.,  574.,  638.,  903.,  872.,  799.,
          896.,  902., 1103.,  632.,  770.,  857.,  933.,  159., 1101.,  498.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]])
torch.Size([5, 20])
tensor([[  99., 1033.,   99., 1033.,    0.],
        [  89.,  803.,   89.,  803.,    0.],
        [  67.,  367.,   67.,  367.,    0.],
        [  75.,  660.,   75.,  660.,    0.],
    

In [89]:
# substrate_features = torch.reshape(substrate_features, (-1,))
# print(substrate_features.shape)
# print(substrate_features)

In [90]:
# vnr_cpu = torch.tensor([10])
# vnr_bw = torch.tensor([30])
# pending = torch.tensor([2])
# substrate_features = torch.cat((substrate_features, vnr_cpu, vnr_bw, pending), 0)

# substrate_features
# substrate_features.shape

In [91]:
print(substrate_features)

tensor([[  99., 1033.,   99., 1033.,    0.],
        [  89.,  803.,   89.,  803.,    0.],
        [  67.,  367.,   67.,  367.,    0.],
        [  75.,  660.,   75.,  660.,    0.],
        [  69.,  634.,   69.,  634.,    0.],
        [  76.,  574.,   76.,  574.,    0.],
        [  85.,  638.,   85.,  638.,    0.],
        [  76.,  903.,   76.,  903.,    0.],
        [  62.,  872.,   62.,  872.,    0.],
        [  68.,  799.,   68.,  799.,    0.],
        [  53.,  896.,   53.,  896.,    0.],
        [ 100.,  902.,  100.,  902.,    0.],
        [  90., 1103.,   90., 1103.,    0.],
        [  87.,  632.,   87.,  632.,    0.],
        [  99.,  770.,   99.,  770.,    0.],
        [  65.,  857.,   65.,  857.,    0.],
        [  65.,  933.,   65.,  933.,    0.],
        [  82.,  159.,   82.,  159.,    0.],
        [  73., 1101.,   73., 1101.,    0.],
        [  76.,  498.,   76.,  498.,    0.]])


- Using 'from_networkx'
    - transfer the torch_geometric

In [92]:
data = from_networkx(net)

In [93]:
print(data)

Data(edge_index=[2, 200], CPU=[20], LOCATION=[20], bandwidth=[200], num_nodes=20)


### Graph Convolution Network
- Generate the GCN class

In [94]:
from torch.nn import Linear
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        # in_channels: 节点特征数   out_channels: 输出的节点分类数
        self.conv1 = GCNConv(in_channels=5, out_channels=4)
        self.conv2 = GCNConv(in_channels=4, out_channels=4)
        self.conv3 = GCNConv(in_channels=4, out_channels=1)
        self.classifier = Linear(1, 20)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.
        
        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(5, 4)
  (conv2): GCNConv(4, 4)
  (conv3): GCNConv(4, 1)
  (classifier): Linear(in_features=1, out_features=20, bias=True)
)


In [96]:
model = GCN()

print(substrate_features.shape, data.edge_index.shape)
print(substrate_features)

out, embedding = model(substrate_features, data.edge_index)
# out, embedding = model(data.x, data.edge_index)
print(f'Embedding shape: {list(embedding.shape)}')

torch.Size([20, 5]) torch.Size([2, 200])
tensor([[  99., 1033.,   99., 1033.,    0.],
        [  89.,  803.,   89.,  803.,    0.],
        [  67.,  367.,   67.,  367.,    0.],
        [  75.,  660.,   75.,  660.,    0.],
        [  69.,  634.,   69.,  634.,    0.],
        [  76.,  574.,   76.,  574.,    0.],
        [  85.,  638.,   85.,  638.,    0.],
        [  76.,  903.,   76.,  903.,    0.],
        [  62.,  872.,   62.,  872.,    0.],
        [  68.,  799.,   68.,  799.,    0.],
        [  53.,  896.,   53.,  896.,    0.],
        [ 100.,  902.,  100.,  902.,    0.],
        [  90., 1103.,   90., 1103.,    0.],
        [  87.,  632.,   87.,  632.,    0.],
        [  99.,  770.,   99.,  770.,    0.],
        [  65.,  857.,   65.,  857.,    0.],
        [  65.,  933.,   65.,  933.,    0.],
        [  82.,  159.,   82.,  159.,    0.],
        [  73., 1101.,   73., 1101.,    0.],
        [  76.,  498.,   76.,  498.,    0.]])
Embedding shape: [20, 1]


In [97]:
print(embedding)
print(embedding.shape)

tensor([[0.7949],
        [0.7536],
        [0.6288],
        [0.7303],
        [0.7094],
        [0.7234],
        [0.7230],
        [0.8206],
        [0.7859],
        [0.7537],
        [0.7836],
        [0.7870],
        [0.8120],
        [0.7117],
        [0.7662],
        [0.7857],
        [0.7831],
        [0.5692],
        [0.8311],
        [0.6881]], grad_fn=<TanhBackward0>)
torch.Size([20, 1])


In [98]:
print(out.shape)
print(out)

torch.Size([20, 20])
tensor([[-0.9015,  0.0903,  0.1798, -0.8786,  0.4872, -1.5868,  0.3313, -0.5921,
         -1.6528, -0.0988, -0.5320, -0.9747,  0.2216,  0.6303,  0.3589, -0.2516,
         -0.3022, -0.4172, -0.3885,  0.6563],
        [-0.9013,  0.0649,  0.2164, -0.8465,  0.4948, -1.5459,  0.3257, -0.5628,
         -1.6157, -0.0820, -0.5099, -0.9745,  0.1939,  0.6404,  0.3506, -0.2702,
         -0.3133, -0.4192, -0.3953,  0.6319],
        [-0.9007, -0.0118,  0.3269, -0.7498,  0.5176, -1.4221,  0.3088, -0.4744,
         -1.5036, -0.0313, -0.4430, -0.9738,  0.1103,  0.6708,  0.3258, -0.3266,
         -0.3469, -0.4254, -0.4156,  0.5585],
        [-0.9012,  0.0506,  0.2371, -0.8284,  0.4991, -1.5227,  0.3226, -0.5463,
         -1.5947, -0.0725, -0.4973, -0.9744,  0.1782,  0.6461,  0.3460, -0.2808,
         -0.3196, -0.4204, -0.3991,  0.6182],
        [-0.9011,  0.0378,  0.2555, -0.8123,  0.5029, -1.5021,  0.3197, -0.5315,
         -1.5760, -0.0641, -0.4862, -0.9743,  0.1643,  0.6511,  0.

# A3C Code
- Simple A3C code

In [ ]:
from torch import nn
import torch
import numpy as np


def v_wrap(np_array, dtype=np.float32):
    if np_array.dtype != dtype:
        np_array = np_array.astype(dtype)
    return torch.from_numpy(np_array)


def set_init(layers):
    for layer in layers:
        nn.init.normal_(layer.weight, mean=0., std=0.1)
        nn.init.constant_(layer.bias, 0.)


def push_and_pull(opt, lnet, gnet, done, s_, bs, buffer_action, buffer_reward, gamma):
    if done:
        v_s_ = 0.               # terminal
    else:
        v_s_ = lnet.forward(v_wrap(s_[None, :]))[-1].data.numpy()[0, 0]

    buffer_v_target = []
    for r in buffer_reward[::-1]:    # reverse buffer r
        v_s_ = r + gamma * v_s_
        buffer_v_target.append(v_s_)
    buffer_v_target.reverse()
    loss = lnet.loss_func(
        v_wrap(np.vstack(bs)),
        v_wrap(np.array(buffer_action), dtype=np.int64) if buffer_action[0].dtype == np.int64 else v_wrap(np.vstack(buffer_action)),
        v_wrap(np.array(buffer_v_target)[:, None]))

    # calculate local gradients and push local parameters to global
    opt.zero_grad()
    loss.backward()
    for lp, gp in zip(lnet.parameters(), gnet.parameters()):
        gp._grad = lp.grad
    opt.step()

    # pull global parameters
    lnet.load_state_dict(gnet.state_dict())


def record(global_ep, global_ep_r, ep_r, res_queue, name):
    with global_ep.get_lock():
        global_ep.value += 1
    with global_ep_r.get_lock():
        if global_ep_r.value == 0.:
            global_ep_r.value = ep_r
        else:
            global_ep_r.value = global_ep_r.value * 0.99 + ep_r * 0.01
    res_queue.put(global_ep_r.value)
    print(
        name,
        "Ep:", global_ep.value,
        "| Ep_r: %.0f" % global_ep_r.value,
    )

In [ ]:
class SharedAdam(torch.optim.Adam):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.99), eps=1e-8,
                 weight_decay=0):
        super(SharedAdam, self).__init__(params, lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        # State initialization
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'] = 0
                state['exp_avg'] = torch.zeros_like(p.data)
                state['exp_avg_sq'] = torch.zeros_like(p.data)

                # share in memory
                state['exp_avg'].share_memory_()
                state['exp_avg_sq'].share_memory_()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import gym
import os
os.environ["OMP_NUM_THREADS"] = "1"

UPDATE_GLOBAL_ITER = 5
GAMMA = 0.9
MAX_EP = 3000

env = gym.make('CartPole-v0')
N_S = env.observation_space.shape[0]
N_A = env.action_space.n

print(N_S, N_A)

In [ ]:
class Net(nn.Module):
    def __init__(self, s_dim, a_dim):
        super(Net, self).__init__()
        self.s_dim = s_dim
        self.a_dim = a_dim
        self.pi1 = nn.Linear(s_dim, 128)
        self.pi2 = nn.Linear(128, a_dim)
        self.v1 = nn.Linear(s_dim, 128)
        self.v2 = nn.Linear(128, 1)
        set_init([self.pi1, self.pi2, self.v1, self.v2])
        self.distribution = torch.distributions.Categorical

    def forward(self, x):
        pi1 = torch.tanh(self.pi1(x))
        logits = self.pi2(pi1)
        v1 = torch.tanh(self.v1(x))
        values = self.v2(v1)
        return logits, values

    def choose_action(self, s):
        self.eval()
        logits, _ = self.forward(s)
        prob = F.softmax(logits, dim=1).data
        m = self.distribution(prob)
        return m.sample().numpy()[0]

    def loss_func(self, s, a, v_t):
        self.train()
        logits, values = self.forward(s)
        td = v_t - values
        c_loss = td.pow(2)
        
        probs = F.softmax(logits, dim=1)
        m = self.distribution(probs)
        exp_v = m.log_prob(a) * td.detach().squeeze()
        a_loss = -exp_v
        total_loss = (c_loss + a_loss).mean()
        return total_loss

In [ ]:
class Worker(mp.Process):
    def __init__(self, gnet, opt, global_ep, global_ep_r, res_queue, name):
        super(Worker, self).__init__()
        self.name = 'w%02i' % name
        self.g_ep, self.g_ep_r, self.res_queue = global_ep, global_ep_r, res_queue
        self.gnet, self.opt = gnet, opt
        self.lnet = Net(N_S, N_A)           # local network
        self.env = gym.make('CartPole-v0').unwrapped

    def run(self):
        total_step = 1
        while self.g_ep.value < MAX_EP:
            s = self.env.reset()
            buffer_s, buffer_action, buffer_reward = [], [], []
            ep_r = 0.
            while True:
#                 if self.name == 'w00':
#                     self.env.render()
                print(s)
                a = self.lnet.choose_action(v_wrap(s[None, :]))
                print(s[None, :])
                print(v_wrap(s[None, :]))
                s_, r, done, _ = self.env.step(a)
                if done: r = -1
                ep_r += r
                buffer_action.append(a)
                buffer_s.append(s)
                buffer_reward.append(r)

                if total_step % UPDATE_GLOBAL_ITER == 0 or done:  # update global and assign to local net
                    # sync
                    push_and_pull(self.opt, self.lnet, self.gnet, done, s_, buffer_s, buffer_action, buffer_reward, GAMMA)
                    buffer_s, buffer_action, buffer_reward = [], [], []

                    if done:  # done and print information
                        record(self.g_ep, self.g_ep_r, ep_r, self.res_queue, self.name)
                        break
                s = s_
                total_step += 1
        self.res_queue.put(None)

In [ ]:
gnet = Net(N_S, N_A)        # global network
gnet.share_memory()         # share the global parameters in multiprocessing
opt = SharedAdam(gnet.parameters(), lr=1e-4, betas=(0.92, 0.999))      # global optimizer
global_ep, global_ep_r, res_queue = mp.Value('i', 0), mp.Value('d', 0.), mp.Queue()
print(mp.cpu_count())

# parallel training
workers = [Worker(gnet, opt, global_ep, global_ep_r, res_queue, i) for i in range(mp.cpu_count())]
[w.start() for w in workers]
res = []                    # record episode reward to plot
while True:
    r = res_queue.get()
    if r is not None:
        res.append(r)
    else:
        break
[w.join() for w in workers]

import matplotlib.pyplot as plt
plt.plot(res)
plt.ylabel('Moving average ep reward')
plt.xlabel('Step')
plt.show()